In [1]:
# giga_takeoutdf_w_jobgroup.pickle
import pickle
with open('giga_takeoutdf_w_jobgroup.pickle', 'rb') as f:
    takeoutdf = pickle.load(f)

In [2]:
# giga_training_features_w_jobgroup.pickle
with open('giga_training_features_w_jobgroup.pickle', 'rb') as f:
    training_features = pickle.load(f)

In [3]:
l1 = set(takeoutdf.columns.tolist())

In [4]:
l2 = set(training_features)

In [5]:
l1-l2

{'HeartBeatTime',
 'JobID',
 'RescheduleCounter',
 'Status_Failed',
 'WallClockTime'}

In [6]:
setofjobids = set(takeoutdf['JobID'].tolist())
listofjobids= list(setofjobids)

In [7]:
min_samples = 0

In [8]:
df_grouped = takeoutdf.groupby(['JobID']).size().reset_index(name='counts')

In [9]:
selectedjobids = df_grouped[df_grouped.counts>min_samples].JobID.tolist()

In [10]:
len(selectedjobids), len(listofjobids)

(320730, 320730)

In [11]:
import os
import gc
import pandas as pd
import numpy as np
import sys
from os import listdir
from os.path import isfile, join
from keras.models import Model, Sequential
from keras.layers import LSTM, Dense, Embedding
from keras.utils import multi_gpu_model
from sklearn.metrics import precision_recall_fscore_support as score
import multiprocessing
from joblib import Parallel, delayed

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [12]:
# FIX 1 of 3 : set the following vars
path                 = "." #path to dir where test pickles reside and h5 file is
modelfingerprint     = "kubernetes_train_[80-20]_giga_long_mltithrdedgentr_PikTranSav_Mdl_[J_Filternet_tstep1]_epch_75_kube_binar_crsentr_adm_binar_acc.py" # name of py file
modelweightsfilename = "kubernetes_train_[80-20]_giga_long_mltithrdedgentr_PikTranSav_Mdl_[J_Filternet_tstep1]_epch_75_kube_binar_crsentr_adm_binar_acc.h5" # name of h5 file

# FIX 2 of 3: update the input_length as per timesteps
input_length    = 1 # 1 or 2                        # X_final.shape[1]
input_dim       = 3541                              # X_final.shape[2]
output_dim      = 1

pickgpus = "0,1"
os.environ["CUDA_VISIBLE_DEVICES"] = pickgpus

def create_model(input_dim, input_length, output_dim):
    print ('Creating model...')
    # FIX 3 of 3: copy everything from here till 'return parallel_model'
    model = Sequential()
    model.add(LSTM(2000, input_shape=(input_length,input_dim),return_sequences=True))
    model.add(LSTM(2000, input_shape=(input_length,input_dim),return_sequences=True))
    model.add(LSTM(1000, input_shape=(input_length,input_dim),return_sequences=True))
    model.add(LSTM(2000, input_shape=(input_length,input_dim),return_sequences=True))
    model.add(LSTM(2000))
    model.add(Dense(output_dim, activation='sigmoid'))
    ###
    print('Initiating parallel GPU model')
    parallel_model = multi_gpu_model(model, gpus=1+pickgpus.count(","))
    print ('Compiling...')
    parallel_model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['binary_accuracy'])
    return parallel_model

In [13]:
# STEP 1
model   = create_model(input_dim, input_length, output_dim)
# STEP 2
print('Loading Model weights before Testing')
model.load_weights( join(path, modelweightsfilename) )

Creating model...
Initiating parallel GPU model
Compiling...
Loading Model weights before Testing


In [14]:
def applyParallel(dfGrouped, func):
    #removed the parallel part since 
    #parallelism is at higher level - at jobID level
    retLst = []
    
    for name, group in dfGrouped:
        retLst.append(func(group))
                                   
    return retLst

In [15]:
with open('giga_training_features_w_jobgroup.pickle', 'rb') as handle:
    train_feat = pickle.load(handle)
    
label_col  = ['Status_Failed']
thresholdd = 1

In [16]:
def samples_label(df_input):
    # 
    k = thresholdd
    input_cols = label_col
    
    # takes a df
    # Put your inputs into a single list
    
    df = pd.DataFrame()
    
    df['single_input_vector'] = df_input[input_cols].apply(tuple, axis=1).apply(list)
    
    # Double-encapsulate list so that you can sum it in the next step and keep time steps as separate elements
    df['single_input_vector'] = df.single_input_vector.apply(lambda x: [list(x)])
        
    # The starting point
    df['cumulative_input_vectors'] = df['single_input_vector'].shift(0)
    
    for i in range(1,k):
        df['cumulative_input_vectors'] += df['single_input_vector'].shift(i)
          
    df.dropna(inplace=True)     # does operation in place & returns None

    # Extract your training data
    X_ = np.asarray(df.cumulative_input_vectors)
    
    # Use hstack to and reshape to make the inputs a 3d vector
    X = np.vstack(X_).reshape(len(df), k, len(input_cols))
    
    # Clean up
    del df
    
    return X
    # returns 3D array

In [17]:
def samples_features(df_input):
    # 
    k = thresholdd
    input_cols = train_feat
    
    # takes a df
    # Put your inputs into a single list
    
    df = pd.DataFrame()
    
    df['single_input_vector'] = df_input[input_cols].apply(tuple, axis=1).apply(list)
    
    # Double-encapsulate list so that you can sum it in the next step and keep time steps as separate elements
    df['single_input_vector'] = df.single_input_vector.apply(lambda x: [list(x)])
        
    # The starting point
    df['cumulative_input_vectors'] = df['single_input_vector'].shift(0)
    
    for i in range(1,k):
        df['cumulative_input_vectors'] += df['single_input_vector'].shift(i)
          
    df.dropna(inplace=True)     # does operation in place & returns None

    # Extract your training data
    X_ = np.asarray(df.cumulative_input_vectors)
    
    # Use hstack to and reshape to make the inputs a 3d vector
    X = np.vstack(X_).reshape(len(df), k, len(input_cols))
    
    # Clean up
    del df
    
    return X
    # returns 3D array

In [18]:
from threading import Lock

def createtensor(df):
    #
    cleanedgrouped = df.groupby('JobID')
    # print('Collectrain started...')
    collecttrain   = applyParallel(cleanedgrouped, samples_features)

    #
    X = []
    for x in collecttrain:
        #len(x)
        for i in x:
            X.append(i)
    del collecttrain
    X = np.array(X)
    
    # print('Collectlabel started...')
    collectlabel = applyParallel(cleanedgrouped, samples_label)
    del cleanedgrouped

    Y=[]
    for x in collectlabel:
        #len(x)
        for i in x:
            Y.append(i)
    del collectlabel
    
    Y  = np.array(Y)
    YY = np.array([x[0][0] for x in Y]).reshape(len(Y),1)
    # 
    X_test, Y_test = X,YY # no shuffling in test set
    #
    return X_test, Y_test

In [19]:
# parameters
samplefrac         = 0.1
decision_threshold = 0.95
minsamplestopick   = 3
lock               = Lock()
global_dict = {}

def g(jobID):
    global model
    global samplefrac
    global global_dict
    global lock
    global takeoutdf
    global minsamplestopick
    
    localsamplefrac = samplefrac
    Y_true = []
    Y_pred = []
    #
    #
    # Sort by timestamp
    getsamples = takeoutdf[takeoutdf.JobID == jobID].copy(deep=True)
    df         = getsamples.sort_values(by=['HeartBeatTime'])
    
    # create 1 timestep 3D tensor
    
    X_test, Y_test     = createtensor(df)
    del df
    del getsamples
    global_dict[jobID] = [X_test, Y_test]
    
    # model is global
    
    # get prediction on each timestep
    
    try:
        lock.acquire()
        y_pred_this_file = model.predict(X_test)
    finally:
        lock.release()
    
    Y_pred.extend([ np.rint(jj[0]) for jj in y_pred_this_file ])
    Y_true.extend([ xj[0] for xj in Y_test ])
    
    #test 1
    if not all(x == Y_true[0] for x in Y_true):
        print('All Y_true labels for jobID %d are not same \n' % jobID)

    truelabel = Y_true[0]
    
    # perform job-level prediction in a while loop:
    firstN         = min( max(minsamplestopick, int(localsamplefrac * len(Y_true))), len(Y_pred) )
    predictedlabel = -1
    
    while(predictedlabel == -1 and localsamplefrac <= 1.0):
        localsamplefrac = (firstN*1.0) / len(Y_pred)
        take            = Y_pred[:firstN]
        
        if take.count(1.0) > decision_threshold * len(take):
            predictedlabel = 1
            break
            
        if take.count(0.0) > decision_threshold * len(take):
            predictedlabel = 0
            break
        
        firstN          = firstN + 1
        
    
    # create a python list that will be return
    # send back
    totaljobduration   = None
    predictionduration = None
     
    return [jobID, totaljobduration, predictionduration, localsamplefrac, len(Y_true), decision_threshold, truelabel, predictedlabel]

In [20]:
len(selectedjobids), len(listofjobids)

(320730, 320730)

In [21]:
# warm up
# https://github.com/keras-team/keras/issues/6124
# Keras builds the GPU function the first time you call predict().

for x in selectedjobids[:10]:
    print(g(x))

[23141083, None, None, 1.0, 1, 0.95, 1, 1]
[23205649, None, None, 0.13043478260869565, 23, 0.95, 1, 1]
[23205658, None, None, 0.15789473684210525, 19, 0.95, 1, 1]
[23205662, None, None, 0.11538461538461539, 26, 0.95, 1, 1]
[23205666, None, None, 0.13636363636363635, 22, 0.95, 1, 1]
[23205667, None, None, 0.14285714285714285, 21, 0.95, 1, 1]
[23205673, None, None, 0.15789473684210525, 19, 0.95, 1, 1]
[23205675, None, None, 0.11538461538461539, 26, 0.95, 1, 1]
[23205692, None, None, 0.16666666666666666, 18, 0.95, 1, 1]
[23205742, None, None, 0.15, 20, 0.95, 1, 1]


In [22]:
def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

In [23]:
length = len(selectedjobids)
breakeachinto  = 30
global_results = []

In [24]:
%%time
import gc

# Test in batches:

for bat in batch([x for x in range(160365, length)], int(length/breakeachinto)):
    global_dict = {}
    
    print("New Batch started : %d to %d " % (bat[0], bat[-1]))
    results = Parallel(n_jobs=48, verbose=1, backend="threading")(map(delayed(g), [selectedjobids[i] for i in bat] ))
    global_results.extend(results)
    del results
    
    # save dict to disk
    start    = bat[0]
    end      = bat[-1]
    filename = 'dict_jobID_Xtest_Ytest_'+str(start)+'_'+str(end)+'_FROM_giga_takeoutdf_w_jobgroup.pickle'
    with open(filename, 'wb') as handle:
        pickle.dump(global_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    del global_dict
    gc.collect()

New Batch started : 160365 to 171055 


[Parallel(n_jobs=48)]: Done 104 tasks      | elapsed:    7.4s
[Parallel(n_jobs=48)]: Done 354 tasks      | elapsed:   20.6s
[Parallel(n_jobs=48)]: Done 704 tasks      | elapsed:   39.4s
[Parallel(n_jobs=48)]: Done 1154 tasks      | elapsed:  1.1min
[Parallel(n_jobs=48)]: Done 1704 tasks      | elapsed:  1.5min
[Parallel(n_jobs=48)]: Done 2354 tasks      | elapsed:  2.1min
[Parallel(n_jobs=48)]: Done 3104 tasks      | elapsed:  2.8min
[Parallel(n_jobs=48)]: Done 3954 tasks      | elapsed:  3.5min
[Parallel(n_jobs=48)]: Done 4904 tasks      | elapsed:  4.4min
[Parallel(n_jobs=48)]: Done 5954 tasks      | elapsed:  5.3min
[Parallel(n_jobs=48)]: Done 7104 tasks      | elapsed:  6.3min
[Parallel(n_jobs=48)]: Done 8354 tasks      | elapsed:  7.4min
[Parallel(n_jobs=48)]: Done 9704 tasks      | elapsed:  8.6min
[Parallel(n_jobs=48)]: Done 10691 out of 10691 | elapsed:  9.4min finished


New Batch started : 171056 to 181746 


[Parallel(n_jobs=48)]: Done 104 tasks      | elapsed:    8.8s
[Parallel(n_jobs=48)]: Done 354 tasks      | elapsed:   22.7s
[Parallel(n_jobs=48)]: Done 704 tasks      | elapsed:   41.4s
[Parallel(n_jobs=48)]: Done 1154 tasks      | elapsed:  1.1min
[Parallel(n_jobs=48)]: Done 1704 tasks      | elapsed:  1.7min
[Parallel(n_jobs=48)]: Done 2354 tasks      | elapsed:  2.3min
[Parallel(n_jobs=48)]: Done 3104 tasks      | elapsed:  3.0min
[Parallel(n_jobs=48)]: Done 3954 tasks      | elapsed:  3.8min
[Parallel(n_jobs=48)]: Done 4904 tasks      | elapsed:  4.7min
[Parallel(n_jobs=48)]: Done 5954 tasks      | elapsed:  5.6min
[Parallel(n_jobs=48)]: Done 7104 tasks      | elapsed:  6.9min
[Parallel(n_jobs=48)]: Done 8354 tasks      | elapsed:  8.2min
[Parallel(n_jobs=48)]: Done 9704 tasks      | elapsed:  9.6min
[Parallel(n_jobs=48)]: Done 10691 out of 10691 | elapsed: 10.7min finished


New Batch started : 181747 to 192437 


...

New Batch started : 299348 to 310038 


[Parallel(n_jobs=48)]: Done 104 tasks      | elapsed:    7.6s
[Parallel(n_jobs=48)]: Done 354 tasks      | elapsed:   21.5s
[Parallel(n_jobs=48)]: Done 704 tasks      | elapsed:   42.2s
[Parallel(n_jobs=48)]: Done 1154 tasks      | elapsed:  1.1min
[Parallel(n_jobs=48)]: Done 1704 tasks      | elapsed:  1.7min
[Parallel(n_jobs=48)]: Done 2354 tasks      | elapsed:  2.3min
[Parallel(n_jobs=48)]: Done 3104 tasks      | elapsed:  3.0min
[Parallel(n_jobs=48)]: Done 3954 tasks      | elapsed:  3.8min
[Parallel(n_jobs=48)]: Done 4904 tasks      | elapsed:  4.7min
[Parallel(n_jobs=48)]: Done 5954 tasks      | elapsed:  5.8min
[Parallel(n_jobs=48)]: Done 7104 tasks      | elapsed:  6.9min
[Parallel(n_jobs=48)]: Done 8354 tasks      | elapsed:  8.2min
[Parallel(n_jobs=48)]: Done 9704 tasks      | elapsed:  9.6min
[Parallel(n_jobs=48)]: Done 10691 out of 10691 | elapsed: 10.5min finished


New Batch started : 310039 to 320729 


[Parallel(n_jobs=48)]: Done 104 tasks      | elapsed:    8.0s
[Parallel(n_jobs=48)]: Done 354 tasks      | elapsed:   22.6s
[Parallel(n_jobs=48)]: Done 704 tasks      | elapsed:   44.8s
[Parallel(n_jobs=48)]: Done 1154 tasks      | elapsed:  1.2min
[Parallel(n_jobs=48)]: Done 1704 tasks      | elapsed:  1.7min
[Parallel(n_jobs=48)]: Done 2354 tasks      | elapsed:  2.4min
[Parallel(n_jobs=48)]: Done 3104 tasks      | elapsed:  3.1min
[Parallel(n_jobs=48)]: Done 3954 tasks      | elapsed:  3.9min
[Parallel(n_jobs=48)]: Done 4904 tasks      | elapsed:  4.8min
[Parallel(n_jobs=48)]: Done 5954 tasks      | elapsed:  5.8min
[Parallel(n_jobs=48)]: Done 7104 tasks      | elapsed:  7.0min
[Parallel(n_jobs=48)]: Done 8354 tasks      | elapsed:  8.3min
[Parallel(n_jobs=48)]: Done 9704 tasks      | elapsed:  9.7min
[Parallel(n_jobs=48)]: Done 10691 out of 10691 | elapsed: 10.6min finished


CPU times: user 3h 21min 19s, sys: 21min 33s, total: 3h 42min 53s
Wall time: 2h 42min 55s


In [25]:
Y_true_job      = []
Y_predicted_job = []
averagefrac     = []
nodecision      = []

for r in global_results:
    if r[-1] != -1:
        Y_true_job.append(r[-2])
        Y_predicted_job.append(r[-1])
        averagefrac.append(r[3])
    else:
        nodecision.append(r)

# [jobID, totaljobduration, predictionduration, localsamplefrac, len(df), decision_threshold, truelabel, predictedlabel]

In [26]:
print('Len of Y_true : ', len(Y_true_job))
print('Len of Y_pred : ', len(Y_predicted_job))

print("Overall accuracy on test set: ", np.mean(np.equal(Y_true_job, Y_predicted_job)))

print("Calculating Score: ")
precision, recall, fscore, support = score(Y_true_job, Y_predicted_job)
print('precision    : {}'.format(precision))
print('recall       : {}'.format(recall))
print('fscore       : {}'.format(fscore))
print('support      : {}'.format(support))

print("Confusion matrix:")
print(pd.crosstab(pd.Series(Y_true_job), pd.Series(Y_predicted_job), rownames=['True'], colnames=['Predicted'], margins=True))
print('*********Test Results End**************')

Len of Y_true :  155797
Len of Y_pred :  155797
Overall accuracy on test set:  0.8603952579317958
Calculating Score: 
precision    : [0.86177494 0.85747706]
recall       : [0.92747991 0.7457353 ]
fscore       : [0.89342101 0.79771205]
support      : [98290 57507]
Confusion matrix:
Predicted       0      1     All
True                            
0           91162   7128   98290
1           14622  42885   57507
All        105784  50013  155797
*********Test Results End**************


In [27]:
np.mean(averagefrac)

0.4554412016763692

In [28]:
len(nodecision)

4568

In [30]:
global_results[:50]

[[40387364, None, None, 0.3333333333333333, 9, 0.95, 1, 0],
 [40387402, None, None, 0.3, 10, 0.95, 0, 0],
 [40387420, None, None, 0.75, 4, 0.95, 1, 0],
 [40387451, None, None, 0.2727272727272727, 11, 0.95, 1, 1],
 [40387457, None, None, 0.21428571428571427, 14, 0.95, 0, 0],
 [40387462, None, None, 0.25, 12, 0.95, 1, 1],
 [40387470, None, None, 1.0625, 16, 0.95, 1, -1],
 [40387484, None, None, 0.3, 10, 0.95, 1, 1],
 [40387495, None, None, 0.25, 12, 0.95, 1, 1],
 [40387497, None, None, 0.3333333333333333, 9, 0.95, 1, 1],
 [40387498, None, None, 0.3, 10, 0.95, 0, 0],
 [40387539, None, None, 0.21428571428571427, 14, 0.95, 0, 0],
 [40387559, None, None, 0.25, 12, 0.95, 1, 1],
 [40387567, None, None, 1.0, 2, 0.95, 1, 0],
 [40387590, None, None, 0.2727272727272727, 11, 0.95, 0, 0],
 [40387591, None, None, 0.25, 12, 0.95, 1, 0],
 [40387610, None, None, 0.75, 4, 0.95, 1, 0],
 [40387629, None, None, 1.0909090909090908, 11, 0.95, 1, -1],
 [40387635, None, None, 0.25, 12, 0.95, 1, 1],
 [40387646, 